# 서울 코로나 감염자 시각화

### Data crawling

In [3]:
import urllib.request
url = "http://www.seoul.go.kr/coronaV/coronaStatus.do#status_page_top"
page = urllib.request.urlopen(url) # url에 대한 연결요청
page = page.read() 

In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page,'html.parser')

In [9]:
seoul_gu = soup.find_all('h5',"update-date")[1]
corona_gu_total = seoul_gu.get_text()

In [10]:
corona_gu_total = corona_gu_total.split('(')[1]
corona_gu_total = corona_gu_total.split()[0:25]
corona_gu_total

['강남70,',
 '강동19,',
 '강북8,',
 '강서31,',
 '관악52,',
 '광진12,',
 '구로35,',
 '금천13,',
 '노원25,',
 '도봉14,',
 '동대문34,',
 '동작37,',
 '마포24,',
 '서대문21,',
 '서초40,',
 '성동22,',
 '성북27,',
 '송파42,',
 '양천23,',
 '영등포27,',
 '용산33,',
 '은평28,',
 '종로17,',
 '중구8,',
 '중랑17,']

### 데이터 가공

In [11]:
import pandas as pd

corona_df = pd.DataFrame(corona_gu_total)
corona_df = corona_df.rename(columns={0 : "seoul_gu"})

corona_df["num_corona"] = corona_df["seoul_gu"].str.extract('(\d+)')
corona_df["seoul_gu"] = corona_df["seoul_gu"].str.extract('(\D+)')

corona_df["seoul_gu"] = corona_df["seoul_gu"] + '구'
corona_df["seoul_gu"][23] = '중구'

# corona_df["num_corona"] = pd.to_numeric(corona_df["num_corona"])
# corona_df.head(3)

In [12]:
corona_df

,seoul_gu,num_corona
0,강남구,70
1,강동구,19
2,강북구,8
3,강서구,31
4,관악구,52
5,광진구,12
6,구로구,35
7,금천구,13
8,노원구,25
9,도봉구,14


In [13]:
import json
geo_data = "C:/Users/kjh96/Github/Study_Geodata/seoul_gu.json"

with open(geo_data,encoding="utf-8") as f:
    data = json.loads(f.read())

In [14]:
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.11879551821994, 37.557222485451305],
      [127.12146867175024, 37.55986003393365],
      [127.12435254630417, 37.56144246249796],
      [127.13593925898998, 37.56564793048277],
      [127.14930548011061, 37.56892250303897],
      [127.15511020940411, 37.57093642128295],
      [127.16683184366129, 37.57672487388627],
      [127.17038810813094, 37.576465605301046],
      [127.17607118428914, 37.57678573961056],
      [127.17905504160184, 37.57791388161732],
      [127.17747787800164, 37.57448983055031],
      [127.1781775408844, 37.571481967974336],
      [127.17995281860672, 37.569309661290504],
      [127.18122821955262, 37.56636089217979],
      [127.18169407550688, 37.56286338914073],
      [127.18408792

In [15]:
# 행정구역 경계에 구별 코로나 감염자 수 추가
for i in range(len(corona_df)) : 
    for j in range(len(corona_df)) : 
        if data['features'][i]['properties']["name"] == corona_df["seoul_gu"][j] : 
            data['features'][i]['properties']["corona"] = int(corona_df["num_corona"][j])

### Visualization

In [25]:
# 통합
from mapboxgl.viz import *
from mapboxgl.utils import create_color_stops 
from mapboxgl.utils import create_numeric_stops
import mapboxgl

# 버젼 확인 
mapboxgl.__version__ 

# 토큰 
token = 'pk.eyJ1IjoiZHVjazk2NjciLCJhIjoiY2thNzRiMXhxMGQ5NTJ0cXB0dGpmZ3RrOSJ9.f45On_lrv6Nm4iO7oCg7nw'

# 서울시 중심부의 경도, 위도 입니다. 
center = [126.986, 37.565] 

# 시각화 할 값에 따른 색상의 범주를 지정해줍니다. 
color_breaks = [0, 10, 20, 30, 40, 50] 
color_stops = create_color_stops(color_breaks, colors='BuPu')

# ChoroplethViz 를 그립니다.
viz = ChoroplethViz(access_token=token, data=data, color_property='corona', color_stops=color_stops, center=center, zoom=10)

# 맵을 -15도 만큼 좌우 회전하고, 45도 만큼 상하 회전합니다. 
viz.bearing = -15
viz.pitch = 45

# 각 데이터에 '인구'를 기준으로 height 값을 줍니다.
viz.height_property = 'corona'

# 높이의 값을 '인구' 에 따라 0 ~ 50000 사이의 값을 0 ~ 3000 사이의 값으로 매핑합니다.
numeric_stops = create_numeric_stops([0, 10, 20, 30, 40, 50], 0, 3000)

viz.height_stops = numeric_stops
viz.height_function_type = 'interpolate'

# 맵을 출력합니다. 
viz.show()

In [24]:
from mapboxgl.viz import *
from mapboxgl.utils import create_color_stops 
import mapboxgl

# 버젼 확인 
mapboxgl.__version__ 

# 토큰 
token = 'pk.eyJ1IjoiZHVjazk2NjciLCJhIjoiY2thNzRiMXhxMGQ5NTJ0cXB0dGpmZ3RrOSJ9.f45On_lrv6Nm4iO7oCg7nw'

# 서울시 중심부의 경도, 위도 입니다. 
center = [126.986, 37.565] 

# 시각화 할 값에 따른 색상의 범주를 지정해줍니다. 
color_breaks = [0, 10, 20, 30, 40, 50] 
color_stops = create_color_stops(color_breaks, colors='BuPu')

# ChoroplethViz 를 그립니다.
viz = ChoroplethViz(access_token=token, data=data, color_property='corona', color_stops=color_stops, center=center, zoom=10)

# 맵을 출력합니다. 
viz.show()

In [23]:
from mapboxgl.utils import create_numeric_stops

# 맵을 -15도 만큼 좌우 회전하고, 45도 만큼 상하 회전합니다. 
viz.bearing = -15
viz.pitch = 45

# 각 데이터에 '인구'를 기준으로 height 값을 줍니다.
viz.height_property = 'corona'

# 높이의 값을 '인구' 에 따라 0 ~ 50000 사이의 값을 0 ~ 3000 사이의 값으로 매핑합니다.
numeric_stops = create_numeric_stops([0, 10, 20, 30, 40, 50], 0, 3000)

viz.height_stops = numeric_stops
viz.height_function_type = 'interpolate'

# render again
viz.show()

### 서울 시내 화장실 지도(추가)

In [50]:
geo_data = 'C:/Users/kjh96/Github/Study_Geodata/toilet_seoul.geojson'

In [51]:
with open(geo_data) as f :
    data = json.loads(f.read())

In [64]:
# GraduatedCircleViz
from mapboxgl.utils import ( create_color_stops, create_radius_stops )
center = [126.986, 37.565]
viz = GraduatedCircleViz(data, access_token=token, color_property='이용량',
                         color_stops=create_color_stops([0, 100, 200, 300, 400, 500], colors='BuPu'),
                         radius_property="이용량",
                         radius_stops=create_radius_stops([0, 100, 200, 300, 400, 500], 0, 8),
                         center=center, 
                         zoom=10)
viz.show()

In [69]:
# HeatmapViz
from mapboxgl.utils import ( create_numeric_stops, create_color_stops )
center = [126.986, 37.565]
viz = HeatmapViz(data,
                 access_token=token, 
                 weight_property="이용량",
                 weight_stops=create_numeric_stops([100, 200, 300, 400, 500], 0, 1),
                 color_stops=create_color_stops([0.1, 0.3, 0.5, 0.7, 0.9], colors='BuPu'),
                 radius_stops=create_numeric_stops([8, 10, 12, 14, 16], 3, 12),
                 center=center, zoom=10 )
viz.show()

In [72]:
# ClusteredCircleViz

from mapboxgl.utils import ( create_color_stops, create_radius_stops )
center = [126.986, 37.565]
viz = ClusteredCircleViz(data,
                         access_token=token,
                         color_stops=create_color_stops([100, 200, 300, 400, 500, 1000], 
                         colors='BuPu'),
                         radius_stops=create_radius_stops([0, 100, 200, 300, 400, 500, 1000], 0, 30),
                         cluster_radius=60,
                         center=center,
                         zoom=10 )
viz.show()